# 将LLM 接入 LangChain

## OpenAI接入langchain

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002447FA91210>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002447FA92D40>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [4]:
output = llm.invoke("请你自我介绍一下自己！")

In [5]:
output

AIMessage(content='你好！我是一个AI助手，专门为用户提供各种服务和帮助。我可以回答各种问题，提供信息和建议，还可以进行语言翻译和文本编辑等任务。我还在不断学习和进化，以提供更好的服务。如果你有任何问题或需要帮助，请随时告诉我！', response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 20, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-86f85006-0ca2-4ee4-bbc2-bb87edacb485-0')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

In [7]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

In [8]:
from langchain.prompts.chat import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)
messages

[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.'),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。')]

In [9]:
output  = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, unsure if this was the place.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 95, 'total_tokens': 136}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9559c01-78e3-43d3-8f1d-eda726c40ef7-0')

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, unsure if this was the place.'

## 完整流程


In [11]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, unsure if this was the place.'

In [12]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language":"英文", "output_language":"中文","text": text})

'我扛着比我的身体还重的行李，潜入尼罗河的底部。经过几道闪电的穿越，我看到了一堆光环，不确定这是否就是那个地方。'

##  使用 LangChain 调用百度文心一言

### 自定义 LLM 接入 langchain
在老版本中，LangChain 是不直接支持文心调用的，我们需要自定义一个支持文心模型调用的 LLM。在这里为了向用户展示如何自定义 LLM，我们在《附1.LangChain自定义LLM》中，简述了这种方法，也可参考[源文档](https://python.langchain.com/docs/modules/model_io/llms/custom_llm)。

此处，我们可以直接调用已自定义好的 Wenxin_LLM，具体如何封装 Wenxin_LLM 见`wenxin_llm.py`。

In [13]:
# 需要下载源码
from wenxin_llm import Wenxin_LLM

In [14]:
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
wenxin_api_key = os.environ["QIANFAN_AK"]
wenxin_secret_key = os.environ["QIANFAN_SK"]

In [15]:
llm = Wenxin_LLM(api_key=wenxin_api_key, secret_key=wenxin_secret_key, system="你是一个助手！")

In [16]:
llm.invoke("你好，请你自我介绍一下！")

[INFO] [04-25 15:33:37] openapi_requestor.py:316 [t:20904]: requesting llm api endpoint: /chat/eb-instant
[INFO] [04-25 15:33:37] oauth.py:207 [t:20904]: trying to refresh access_token for ak `nQQgiu***`
[INFO] [04-25 15:33:38] oauth.py:220 [t:20904]: sucessfully refresh access_token


'你好！我是助手，负责协助您完成各种任务。我具备快速学习和处理信息的能力，能够根据您的需求提供帮助和回答问题。无论您需要什么帮助，我都会尽力满足您的需求。'

###  在 langchain 直接调用文心一言

我们也可以使用新版 LangChain，来直接调用文心一言大模型。

In [17]:

from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]

In [7]:
# Install required dependencies
# pip来安装名为langchain和langchain-community的软件包，-q选项表示安静模式，即不输出安装过程中的详细信息。-U选项表示升级已安装的软件包到最新版本。
%pip install -qU langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.llms import QianfanLLMEndpoint

llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

d:\software\miniconda3\envs\llm-universe\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
[WARNING] [04-25 15:34:11] base.py:516 [t:20904]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [04-25 15:34:11] openapi_requestor.py:316 [t:20904]: requesting llm api endpoint: /chat/eb-instant


你好！我是一位喜欢通过文字与人交流的AI助手。我能够帮助你回答各种问题，提供有用的信息，也可以帮助你更深入地了解一些主题。如果你有任何需要，随时都可以向我提问。


## 创建一个LLM

In [19]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("请你自我介绍一下自己！")

AIMessage(content='你好！我是一个AI助手，专门为用户提供各种服务和帮助。我可以回答各种问题，提供信息和建议，还可以进行语言翻译和文本编辑等任务。我还在不断学习和进化，以提供更好的服务。如果你有任何问题或需要帮助，请随时告诉我！', response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 20, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76f10443-d919-43a9-9ff6-d92f22287af0-0')